# Chapter 1

### Exercise 1.1

**Below is a sequence of expressions. What is the result printed by the interpreter in response to each expression? Assume that the sequence is to be evaluated in the order in which it is presented.**

```scheme
>>> 10
10
>>> (+ 5 3 4)
12
>>> (- 9 1)
8
>>> (/ 6 2)
3
>>> (+ (* 2 4) (- 4 6))
6
>>> (define a 3)
>>> (define b (+ a 1))
>>> (+ a b (* a b))
19
>>> (= a b)
F
>>> (if (and (> b a) (< b (* a b)))
        b
        a)
4
>>> (cond ((= a 4) 6)
          ((= b 4) (+ 6 7 a))
          (else 25))
16
>>> (+ 2 (if (> b a) b a))
6
>>> (* (cond ((> a b) a)
             ((< a b) b)
             (else -1))
       (+ a 1))
16      
```

### Exercise 1.2

**Translate the following expression into prefix form:**
$$\frac{5+4+(2-(3-(6+\frac{4}{5})))}{3(6-2)(2-7)}$$

```scheme
(/ (+ 5 4 (- 2 (- 3 (+ 6 (/ 4 5)))))
   (* 3 (- 6 2) (- 2 7)))

```

### Exercise 1.3

**Define a procedure that takes three numbers
as arguments and returns the sum of the squares of the two
larger numbers.**

In [26]:
(define (sum-squares a b)
  (+ (* a a) (* b b)))
(define (sum-squares-of-large-two a b c)
  (cond ((and (< a b) (< a c)) (sum-squares b c))
        ((and (< b a) (< b c)) (sum-squares a c))
        (else (sum-squares a b))))
(sum-squares-of-large-two 2 3 4)

25

### Exercise 1.4

**Observe that our model of evaluation allows
for combinations whose operators are compound expressions.
Use this observation to describe the behavior of the
following procedure:**
```scheme
(define (a-plus-abs-b a b)
((if (> b 0) + -) a b))
```

The procudure `a-plus-abs-b` performs the mathematical operation `a + |b|`. That is, it adds the absolute value of b to a. That is because the operator to apply to a and b is decided by the if-expression, which uses  + if b is greater than 0 and - otherwise.

### Exercise 1.5

**Ben Bitdiddle has invented a test to determine whether the interpreter he is faced with is using applicative order evaluation or normal-order evaluation. He defines the following two procedures:**
```scheme
(define (p) (p))
(define (test x y)
(if (= x 0) 0 y))
```
**Then he evaluates the expression**
```scheme
(test 0 (p))
```
**What behavior will Ben observe with an interpreter that uses applicative-order evaluation? What behavior will he observe with an interpreter that uses normal-order evaluation? Explain your answer. (Assume that the evaluation rule for the special form if is the same whether the interpreter is using normal or applicative order: The predicate expression is evaluated first, and the result determines whether to evaluate the consequent or the alternative expression.)**

In an applicative-order interpreter, the interpreter will get stuck trying to evaluate the expression. That is because an applicative-order evaluator will try to evaluate the arguments of `test` before substituting them into the definition. This means trying to evaluate `p`, which is infinitely recursive.

In a normal-order interpreter, Ben will observe that the expression evaluates to 0. That is because the interpreter will first substitute the arguments of `test` into its definition and then evaluate the `if` expression in the definition. As the first parameter is 0, the if expression returns 0 without ever evaluating `p`.

### Exercise 1.6

**Alyssa P. Hacker doesn’t see why `if` needs to be provided as a special form. “Why can’t I just define it as an ordinary procedure in terms of `cond`?” she asks. Alyssa’s friend Eva Lu Ator claims this can indeed be done, and she defines a new version of if:**
```scheme
(define (new-if predicate then-clause else-clause)
(cond (predicate then-clause)
(else else-clause)))
```
**Eva demonstrates the program for Alyssa:**
```scheme
(new-if (= 2 3) 0 5)
5
(new-if (= 1 1) 0 5)
0
```
**Delighted, Alyssa uses `new-if` to rewrite the square-root
program:**
```scheme
(define (sqrt-iter guess x)
(new-if (good-enough? guess x)
guess
(sqrt-iter (improve guess x) x)))
```
**What happens when Alyssa attempts to use this to compute square roots? Explain.**

When Alyssa atttempts to use this to compute square roots, the evalution gets stuck in infinite recursion. That is because `new-if` is a function instead of a special form, and that means in an applicative-order interpreter all of its arguments will be evaluated before they are substituted in the definition. In this case, the `new-if` call will need to evaluate `sqrt-iter` every time. This leads to infinite recursion with nowhere to stop.

### Exercise 1.7

**The `good-enough?` test used in computing square roots will not be very effective for finding the square roots of very small numbers. Also, in real computers, arithmetic operations are almost always performed with limited precision. This makes our test inadequate for very large numbers. Explain these statements, with examples showing how the test fails for small and large numbers. An alternative strategy for implementing `good-enough?` is to watch how `guess` changes from one iteration to the next and to stop when the change is a very small fraction of the guess. Design a square-root procedure that uses this kind of end test. Does this work better for small and large numbers?**

Our good-enough? test compares the square of the guess with our radicand and returns true if the difference is lower than a certain fixed value. $0.001$ is the value used in the text. This creates problems for very small numbers because within a few iterations, the guess will be smaller than or close to $0.001$, and that guess will be returned as the square root even though, compared to the actual square root, it is not very close. For example, the square root of $0.00000004$  is $0.0002$. But if a guess is 0.0008, which is eight times the actual square root, it will be good enough for our test becuase the difference is $0.0008 - 0.0002 = 0.0006 \lt 0.001 $

Also, our test is inadequate for very large numbers because arithmetic operations are almost always performed with limited precision. Floating point numbers are stored with a limited, fixed number of bits for the significant digits and a limited, fixed number of bits for the exponent. When a number gets really large, even changing the least significant bit on by one changes the number by a large difference. As a result, limited precision floating point numbers cannot represent a lot of the intermediate values between large numbers that it can represent. So, for large enough numbers, our computations may never produce guesses that are within $1$ of the true square root, let alone $0.001$ of the square root.

In [27]:
(define (average x y)
  (/ (+ x y) 2))

(define (square x) (* x x))

(define (good-enough? oldGuess newGuess)
  (< (abs (/ (- newGuess oldGuess) oldGuess)) 0.00001))

(define (improve guess x)
  (average guess (/ x guess)))

(define (sqrt-iter guess x)
  (if (good-enough? guess (improve guess x))
      guess
      (sqrt-iter (improve guess x) x)))

(sqrt-iter 1.0 0.0064)

0.08000000019530852

In [28]:
(sqrt-iter 1.0 10000)

100.00000025490743

Yes, this square root procedure with the improved good-enough? test works much better for large and small numbers.

### Exercise 1.8

**Newton’s method for cube roots is based on the fact that if $y$ is an approximation to the cube root of $x$, then a better approximation is given by the value**

$$\frac{x/y^2 + 2y}{3}$$

**Use this formula to implement a cube-root procedure analogous to the square-root procedure.**

In [29]:
(define (improve guess x)
  (/ (+ (/ x (* guess guess)) (* 2 guess)) 3))

(define (cbrt-iter guess x)
  (if (good-enough? guess (improve guess x))
      guess
      (cbrt-iter (improve guess x) x)))
(define (cbrt x) (cbrt-iter 1.0 x))

(cbrt 27)

3.0000005410641766

### Exercise 1.9

**Each of the following two procedures defines a method for adding two positive integers in terms of the procedures `inc`, which increments its argument by 1, and `dec`, which decrements its argument by 1.**

```scheme
(define (+ a b)
(if (= a 0) b (inc (+ (dec a) b))))
(define (+ a b)
(if (= a 0) b (+ (dec a) (inc b))))
```
**Using the substitution model, illustrate the process generated by each procedure in evaluating `(+ 4 5)`. Are these processes iterative or recursive?**

The first add procedure evolves the following process :
```scheme
(+ 4 5)
(inc (+ 3 5))
(inc (inc (+ 2 5)))
(inc (inc (inc (+ 1 5))))
(inc (inc (inc (inc (+ 0 5)))))
(inc (inc (inc (inc 5))))
(inc (inc (inc 6)))
(inc (inc 7))
(inc 8)
9
```
This process is recursive, because the space required by the procedure grows and shrinks, and at it's highest the call stack goes as deep as the value of the first parameter.

The second add procedure evolves the following process:
```scheme
(+ 4 5)
(+ 3 6)
(+ 2 7)
(+ 1 8)
(+ 0 9)
9
```
This process is iterative, as the space required is constant throughout the life of the process.

### Exercise 1.10

**The following procedure computes a mathematical function called Ackermann’s function.**

In [1]:
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1) (A x (- y 1))))))

**What are the values of the following expressions?**
```scheme
(A 1 10)
(A 2 4)
(A 3 3)
```
**Consider the following procedures, where `A` is the procedure defined above:**
```scheme
(define (f n) (A 0 n))
(define (g n) (A 1 n))
(define (h n) (A 2 n))
(define (k n) (* 5 n n))
```
**Give concise mathematical definitions for the functions computed by the procedures `f`, `g`, and `h` for positive integer values of *n*. For example, `(k n)` computes $5n^2$.**

In [2]:
(A 1 10)

1024

In [3]:
(A 2 4)

65536

In [4]:
(A 3 3)

65536

For `(f n)`, the concise mathematical definition is obviously $2*n$, becasue the second `cond` clause defines `(A x y)` as `(* 2 y)` when `x` is 0.

For `(g n)`, hand substitution on a few test values shows that the mathematical operation performed is exponentiation with base 2. That is, $2^n$

For `(h n)`, similarly hand-substituting shows that it performs repeated exponentiation (tetration). So, the mathematical form is $2^{2^{2...^{2}}} (n\ times)$

Looking at the functions computed by `(A 0 y)`, `(A 1 y)`, `(A 2 y)`, etc, it appears that each function in the series repeatedly applies the previous one on itself `y` times. So they can be represented mathematically in Knuth's Up-Arrow notation:

`(A 0 y)` $ = 2*y$ 

`(A 1 y)` $ = 2\uparrow y$

`(A 2 y)` $ = 2\uparrow \uparrow y$

`(A x y)` $ = 2\uparrow^x y$

### Exercise 1.11

**A function $f$ is defined by the rule that**

$$
f(n) = 
\begin{cases}
n & \text{if} \; n \lt 3\\
f(n-1) + 2 * f(n-2) + 3 * f(n-3) & \text{if} \; n \ge 3
\end{cases}
$$

**Write a procedure that computes $f$ by means of a recursive process. Write a procedure that computes $f$ by means of an iterative process.**


Here is a procedure that computes $f$ by means of a (tree) recursive process:

In [11]:
(define (f n)
  (if (< n 3) n (+ (f (- n 1)) (* 2 (f (- n 2))) (* 3 (f (- n 3))))))

In [13]:
(f 10)

1892

Here is a procedure that computes $f$ by means of an iterative process:

In [13]:
(define (f-iter a b c nn) 
  (cond ((= nn 2) a)
        ((= nn 1) b)
        ((= nn 0) c)
        (else (f-iter (+ a (* 2 b) (* 3 c)) a b (- nn 1)))))
(define (f n) (f-iter 2 1 0 n))

In [14]:
(f 10)

1892

### Exercise 1.12

**The following pattern of numbers is called *Pascal's triangle.***

$$
\quad\quad\quad\quad1\quad\quad\quad\quad\\
\quad\quad\quad1\quad1\quad\quad\quad\\
\quad\quad1\quad2\quad1\quad\quad\\
\quad1\quad3\quad3\quad1\quad\\
1\quad4\quad6\quad4\quad1\\
\quad\quad\quad.\;.\;.\quad\quad\quad\\
$$

**The numbers at the edge of the triangle are all 1, and each number inside the triangle is the sum of the two numbers above it. Write a procedure that computes elements of Pascal's triangle by means of a recursive process.**

In [9]:
(define (pascal m n)
  (cond ((= m n) 1)
        ((= n 0) 1)
        (else (+ (pascal (- m 1) n) (pascal (- m 1) (- n 1))))))

In [12]:
(pascal 4 2)

6

### Exercise 1.13

**Prove that $Fib(n)$ is the closest integer to $\varphi^n / \sqrt{5}$, where $\varphi = (1 + \sqrt(5)) / 2$. Hint: Let $\psi = (1 - \sqrt{5}) / 2$. Use induction and the definition of the Fibonacci numbers to prove that $Fib(n) = (\varphi^n - \psi^n) / \sqrt{5}$.**

First, note that $\varphi^{n+2} = \varphi^{n+1} + \varphi^n$ and that $\varphi + \psi = 1$. These can be easily proved by using the definitions of $\varphi$ and $\psi$.

We have $Fib(0) = 0$ and $Fib(1) = 1$. So the value of $Fib(n)$ is equal to the value of Fibonacci sequence for $n = 0$ and $1$. To use mathematical induction, let us suppose that for all natural numbers up to $n+1$, $Fib(n+1)$ is equal to the $(n+1)^{th}$ fibonacci number. We will show that in that case, $Fib(n+2)$ equals the $(n+2)^{th}$ fibonacci number.

$$
\begin{align}
Fib(n) + Fib(n+1) &= \frac{\varphi^n - \psi^n}{\sqrt{5}} + \frac{\varphi^{n+1} - \psi^{n+1}}{\sqrt{5}}\\
\sqrt{5}(Fib(n) + Fib(n+1)) &= (\varphi^n - \psi^n) + (\varphi^{n+1} - \psi^{n+1})\\
&= (\varphi-\psi)(\varphi^{n-1} + \varphi^{n-2}\psi + ... + \varphi\psi^{n-2} + \psi^{n-1}) + (\varphi-\psi)(\varphi^n + \varphi^{n-1}\psi + ... + \varphi\psi^{n-1} + \psi^n)\\
&= (\varphi-\psi) ((\varphi^{n-1} + \varphi^n) + \psi (\varphi^{n-2} + \varphi^{n-1}) + ... + \psi^{n-1} (\varphi+1) + \psi^n)\\
&= (\varphi-\psi) (\varphi^{n+1} + \psi\varphi^n + ... + \psi^{n-1}\varphi^2 + \psi^n)\\
&= (\varphi-\psi) (\varphi^{n+1} + \psi\varphi^n + ... + \psi^{n-1}\varphi^2 + \psi^n(\varphi + \psi))\\
&= (\varphi-\psi) (\varphi^{n+1} + \psi\varphi^n + ... + \psi^{n-1}\varphi^2 + \psi^n\varphi + \psi^{n+1}))\\
&= (\varphi^{n+2} - \psi^{n+2})\\
Fib(n) + Fib(n+1) &= \frac{\varphi^{n+2} - \psi^{n+2}}{\sqrt{5}}\\
Fib(n) + Fib(n+1) &= Fib(n+2)
\end{align}
$$

We have proved that $Fib(n+2)$ equals the $(n+2)^{th}$ fibonacci number. By the principle of mathematical induction, $Fib(n)$ equals the $n^{th}$ fibonacci number for all  $n \ge 0$.

Now, observe that $|\psi/\sqrt{5}| \lt 0.5$. So, $|\psi^n/\sqrt{5}| \lt 0.5$ for all natural $n \gt 0$. We have $Fib(n) = \varphi^n / \sqrt{5} - \psi^n / \sqrt{5}$. As $Fib(n)$ is an integer, when the second term is is negative, the non-integer parts of the two terms add up. As the non-integer part of any number cannot be greater than $1$ and that of the second term cannot be greater than $0.5$, they must add up to $1$. Because the second term is less than $0.5$, the first term must be greater. Hence, the closest integer to the first term is also the value of $Fib(n)$. When the second term is positive, their non-integer parts subtract. To make the result an integer, they have to be equal. As the second term cannot be greater than $0.5$. the non-integer part of the first term cannot be greater than $0.5$ too. Hence, the closest integer to the first term is also the value of $Fib(n)$.

### Exercise 1.14

**Draw the tree illustrating the process generated by the `count-change` procedure of Section 1.2.2 in making change for 11 cents. What are the orders of growth of space and number of steps used by this process as the amount to be changed increases?**

Here is an image depicting the tree generated by the process. Repeated executions with the same input have been merged in the tree below to keep the size manageable.

<img src="aux/tree-draw/tree.png"/>

The order of growth of the space used by this process is $\Theta(m+n)$ where $m$ is the amount and $n$ is the number of coins. This is because the maximum depth of the tree reaches $\frac{m}{d_0} + n$ where $d_0$ is the smallest denomination. 

*Note: As for the order of growth of steps used by the process, I couldn't find of a straightforward way to calculate it. I read somewhere that you need to use generating functions to solve the recurrence, and I'll update the answer after I study them and find an answer. There were a few answers on the internet but none of them felt convincing or rigorous.*

### Exercise 1.15

**The sine of an angle (specified in radians) can be computed by making use of the approximation $\sin{x} \approx x$ if $x$ is sufficiently small, and the trigonometric identity**

$$
\sin{x} = 3\sin{\frac{x}{3}} - 4\sin^3{\frac{x}{3}}
$$

**to reduce the size of the argument of sin. (For purposes of this exercise an angle is considered "sufficiently small" if its magnitude is not greater than 0.1 radians.) These ideas are incorporated in the following procedures:**

```scheme
(define (cube x) (* x x x))
(define (p x) (- (* 3 x) (* 4 (cube x))))
(define (sine angle)
  (if (not (> (abs angle) 0.1))
      angle
      (p (sine (/ angle 3.0)))))
```

**a. How many times is the procedure `p` applied when `(sine 12.15)` is evaluated?**

**b. What is the order of growth in space and number of steps (as a function of $a$) used by the processs generated by the `sine` procedure when `(sine a)` is evaluated?**

a. When `(sine 12.15)` is evaluated, the procedure `p` is applied 5 times. That is because `p` is applied on every application of `sine` when the argument has an absolute value smaller than $0.1$. Since it takes 5 divisions by 3 for $12.15$ to reach such a value ($0.5$), `p` is applied 5 times. 

b. `sine` is a linear recursive process. Both the space and number of steps required is proportional to the number of times the argument $a$ can be divided by $3$ until the absolute value is less than or equal to $0.1$. That is, $\lceil\log_3{\frac{|a|}{0.1}}\rceil$. As an order of growth in big-oh notation, we can write this as simply $\mathcal{O}(\log{|a|})$. This is the order of growth of both the space and the number of steps.

### Exercise 1.16

**Design a procedure that evolves an iterative exponentiation process that uses successive squaring and uses a logarithmic number of steps, as does `fast-expt`. (Hint: Using the observation that $(b^{n/2})^2 = (b^2)^{n/2}$, keep, along with the exponent $n$ and the base $b$, an additional state variable $a$, and define the state transformation in such a way that the product $ab^n$ is unchanged from state to state. At the beginning of the process $a$ is taken to be $1$, and the answer is given by the value of a at the end of the process. In general, the technique of defining an invariant quantity that remains unchanged from state to state is a powerful way to think about the design of iterative algorithms.)**

Here is a procedure (`exp`) that evolves an iterative process and calculates the exponent $b^n$.

In [2]:
(define (odd? n) (= (remainder n 2) 1))
(define (exp-iter b n a) (if (= n 0) a (exp-iter (* b b) (quotient n 2) (if (odd? n) (* a b) a))))
(define (exp b n) (exp-iter b n 1))

(exp 5 12)

244140625

### Exercicse 1.17

**The exponentiation algorithms in this section are based on performing exponentiation by means of repeated multiplication. In a similar way, one can perform integer multiplication by means of repeated addition. The following multiplication procedure (in which it is assumed that our language can only add, not multiply) is analogous to the `expt` procedure:**

```scheme
(define (* a b)
  (if (= b 0)
      0
      (+ a (* (- b 1)))))
```

**This algorithm takes a number of steps that is linear in `b`. Now suppose we include, together with addition, operations `double`, which doubles an integer, and `halve`, which divides an (even) integer by 2. Using these, design a multiplication procedure analogous to `fast-expt` that uses a logarithmic number of steps.**

`mul` is such a procedure for multiplication:

In [1]:
(define (double n) (+ n n))
(define (halve n) (quotient n 2))
(define (mul a b)
  (cond ((= b 0) 0)
        ((odd? b) (+ a (mul a (- b 1))))
        (else (mul (double a) (halve b)))))

(mul 12 5)

60

### Exercise 1.18

**Using the results of Exercise 1.16 and Exercise 11.7, devise a procedure that generates an iterative process for multiplying two integers in terms of adding, doubling, and halving and uses a logarithmic number of steps.**

The procedure `mul` defined below generates an iterative process for multiplying two integers, using only adding, doubling and halving operations.

In [8]:
(define (double n) (+ n n))
(define (halve n) (quotient n 2))
(define (odd? n) (= (remainder n 2) 1))
(define (mul-iter b n a) (if (= n 0) a (mul-iter (double b) (halve n) (if (odd? n) (+ a b) a))))
(define (mul b n) (mul-iter b n 0))

(mul 12 5)

60

### Exercise 1.19

**There is a clever algorithm for computing the Fibonacci numbers in a logarithmic number of steps. Recall the transformation of the state variables $a$ and $b$ in the `fib-iter` process of Section 1.2.2: $a \leftarrow a+b$ and $b \leftarrow a$. Call this transformation $T$, and observe that applying $T$ over and over again $n$ times, starting with $1$ and $0$, produces the pair $Fib(n + 1)$ and $Fib(n)$. In other words, the Fibonacci numbers are produced by applying $T^n$, the $n^{th}$ power of the transformation $T$, starting with the pair $(1, 0)$. Now consider
$T$ to be the special case of $p = 0$ and $q = 1$ in a family of transformations $T_{pq}$, where $T_{pq}$ transforms the pair $(a,b)$ according to $a \leftarrow bq + aq + ap$ and $b \leftarrow bp + aq$. Show that if we apply such a transformation $T_{pq}$ twice, the effect is the same as using a single transformation $T_{p'q'}$ of the same form, and compute $p'$ and $q'$ in terms of $p$ and $q$. This gives us an explicit way to square these transformations, and thus we can compute $T^n$ using successive squaring, as in the `fast-expt` procedure. Put this all together to complete the following procedure, which runs in a logarithmic number of steps:**

The expressions for $p'$ and $q'$ can be derived from algebraic manipulation of the operations performed by $T_{pq}$. Doing so, we see that $p' = p^2 + q^2$ and $q' = 2pq + q^2$. Plugging in the expressions, we get a working procedure that generates fibonacci numbers in logarithmic number of steps:

In [5]:
(define (fib n)
  (fib-iter 1 0 0 1 n))
(define (fib-iter a b p q count)
  (cond ((= count 0) b)
        ((even? count)
         (fib-iter a
                   b
                   (+ (* p p) (* q q ))
                   (+ (* 2 p q) (* q q))
                   (/ count 2)))
        (else (fib-iter (+ (* b q) (* a q) (* a p))
                        (+ (* b p) (* a q))
                        p
                        q
                        (- count 1)))))
(fib 9)

34

### Exercise 1.20

**The process that a procedure generates is of course dependent on the rules used by the interpreter. As an example, consider the iterative `gcd` procedure given above. Suppose we were to interpret this procedure using normal-order evaluation, as discussed in Section 1.1.5. (The normal-order-evaluation rule for `if` is described in Exercise 1.5.) Using the substitution method (for normal order), illustrate the process generated in evaluating `(gcd 206 40)` and indicate the `remainder` operations that are actually performed. How many `remainder` operations are actually performed in the normal-order evaluation of `(gcd 206 40)`? In the applicative-order evaluation?**

The evolution of the process when `(gcd 206 40)` is evaluated in normal order is shown below:

```scheme
(gcd 206 40)
(if (= 40 0)
    206
    (gcd 40 (remainder 206 40)))
(gcd 40 (remainder 206 40))
(if (= (remainder 206 40) 0)
    40
    (gcd (remainder 206 40) (remainder 40 (remainder 206 40))))
; (remainder 206 40) is evaluated by the implementation of the inbuilt procedure =
; 1 application
(gcd (remainder 206 40) (remainder 40 (remainder 206 40)))
(if (= (remainder 40 (remainder 206 40)))
    (remainder 206 40)
    (gcd (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40))))))
; (remainder 40 (remainder 206 40)) is evaluated by the implementation of the inbuilt procedure =
; 2 applications
(gcd (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40)))))
(if (= (remainder (remainder 206 40) (remainder 40 (remainder 206 40))) 0)
    (remainder 40 (remainder 206 40))
    (gcd (remainder (remainder 206 40) (remainder 40 (remainder 206 40))) (remainder (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40))))))
; (remainder (remainder 206 40) (remainder 40 (remainder 206 40))) is evaluated by the implementation of the inbuilt procedure =
; 4 applications
(gcd (remainder (remainder 206 40) (remainder 40 (remainder 206 40))) (remainder (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40)))))
(if (= (remainder (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40)))) 0)
    (remainder (remainder 206 40) (remainder 40 (remainder 206 40)))
    (gcd (remainder (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40)))) (remainder (remainder (remainder 206 40) (remainder 40 (remainder 206 40))) (remainder (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40)))))))
; (remainder (remainder 40 (remainder 206 40)) (remainder (remainder 206 40) (remainder 40 (remainder 206 40))) is evaluated by the implementation of the inbuilt procedure =
; 7 applications
; This time it is 0
(remainder (remainder 206 40) (remainder 40 (remainder 206 40)))
; Inner applications of remainder are evaluated by the implementation of remainder
; 4 applications
2
```

In the normal-order evaluation, the `remainder` operation is performed a total of 18 times.

The evolution of the process when `(gcd 206 40)` is evaluated in applicative order is shown below:

```scheme
(gcd 206 40)
(if (= 40 0)
    206
    (gcd 40 (remainder 206 40)))
(gcd 40 (remainder 206 40))
(gcd 40 6)
(if (= 6 0)
    40
    (gcd 6 (remainder 40 6)))
(gcd 6 (remainder 40 6))
(gcd 6 4)
(if (= 4 0)
    6
    (gcd 4 (remainder 6 4)))
(gcd 4 (remainder 6 4))
(gcd 4 2)
(if (= 2 0)
    4
    (gcd 2 (remainder 4 2)))
(gcd 2 (remainder 4 2))
(gcd 2 0)
(if (= 0 0)
    2
    (gcd 0 (remainder 2 0)))
2
```

As we can see, in the applicative-order evaluation, the `remainder` operation is performed 4 times.

### Exercise 1.21

**Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: 199, 1999, 19999.**

In [11]:
(define (smallest-divisor n) (find-divisor n 2))
(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))))
(define (divides? a b) (= (remainder b a) 0))
(define (square a) (* a a))

In [4]:
(smallest-divisor 199)

199

In [5]:
(smallest-divisor 1999)

1999

In [6]:
(smallest-divisor 19999)

7

### Exercise 1.22

**Most Lisp implementations include a primitive called `runtime` that returns an integer that specifies the amount of time the system has been running (measured, for example, in microseconds). The following `timed-prime-test` procedure, when called with an integer $n$, prints $n$ and checks to see if $n$ is prime. If $n$ is prime, the procedure prints three asterisks followed by the amount of time used in performing the test.**

In [7]:
(define (timed-prime-test n)
  (newline)
  (display n)
  (start-prime-test n (runtime)))
(define (start-prime-test n start-time)
  (if (prime? n)
      (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
  (display " *** ")
  (display elapsed-time))

**Using this procedure, write a procedure `search-for-primes` that checks the primality of consecutive odd integers in a specified range. Use your procedure to find the three smallest primes larger than $1000$; larger than $10,000$; larger than $100,000$; larger than $1,000,000$. Note the time needed to test each prime. Since the testing algorithm has order of growth of $\Theta(\sqrt{n})$, you should expect that testing for primes around $10,000$ should take about $\sqrt{10}$ times as long as testing for primes around $1000$. Do your timing data bear this out?  How well do the data for $100,000$ and $1,000,000$ support the $\Theta(\sqrt{n})$ prediction? Is your result compatible with the notion that programs on your machine run in time proportional to the number of steps required for the computation?**


In [14]:
(define (prime? n)
  (= n (smallest-divisor n)))

(python-exec 
"
import time
def runtime():
  return int(round(time.time() * 1000000))
")

(define (search-for-primes start end)
  (define (test-and-next test)
    (timed-prime-test test)
    (search-for-primes (+ test 2) end))
  (if (<= start end) 
      (if (= (remainder start 2) 0)
          (search-for-primes (+ start 1) end)
          (test-and-next start))))

In [15]:
(search-for-primes 1000 1020)


1001
1003
1005
1007
1009 *** 34456
1011
1013 *** 40018
1015
1017
1019 *** 31267

#f

In [18]:
(search-for-primes 10000 10038)


10001
10003
10005
10007 *** 91252
10009 *** 108091
10011
10013
10015
10017
10019
10021
10023
10025
10027
10029
10031
10033
10035
10037 *** 90199

#f

In [20]:
(search-for-primes 100000 100048)


100001
100003 *** 324643
100005
100007
100009
100011
100013
100015
100017
100019 *** 379223
100021
100023
100025
100027
100029
100031
100033
100035
100037
100039
100041
100043 *** 336446
100045
100047

#f

In [22]:
(search-for-primes 1000000 1000038)


1000001
1000003 *** 797337
1000005
1000007
1000009
1000011
1000013
1000015
1000017
1000019
1000021
1000023
1000025
1000027
1000029
1000031
1000033 *** 996661
1000035
1000037 *** 844889

#f

If we take the averages of the three runtimes in each run of `search-for-primes`, we see the following numbers:

| Primes around number | Average runtime | Ratio to previous average |
|----------------------|-----------------|---------------------------|
| 1000                 | 35247.0         | -                         |
| 10000                | 96514.0         | 2.738                     |
| 100000               | 346770.67       | 3.593                     |
| 1000000              | 879629.0        | 2.537                     |

As we can see, the ratio of the average runtimes between $10000$ and $1000$, $100000$ and $10000$, and $100000$ and $1000000$ is close to $\sqrt{10}$, whose value is $3.162$. This result is compatible with the notion that programs on a machine run in time proportional to the number of steps required for the computation.

### Exercise 1.23

**The `smallest-divisor` procedure shown at the start of this section does lots of needless testing: After it checks to see if the number is divisible by $2$ there is no point in checking to see if it is divisible by any larger even numbers. This suggests that the values used for `test-divisor` should not be $2, 3, 4, 5, 6,\ldots$, but rather $2, 3, 5, 7, 9,\ldots $. To implement this change, define a procedure next that returns $3$ if its input is equal to $2$ and otherwise returns its input plus $2$. Modify the `smallest-divisor` procedure to use `(next test-divisor)` instead of `(+ test-divisor 1)`. With `timed-prime-test` incorporating this modified version of `smallest-divisor`, run the test for each of the 12 primes found in Exercise 1.22. Since this modification halves the number of test steps, you should expect it to run about twice as fast. Is this expectation confirmed? If not, what is the observed ratio of the speeds of the two algorithms, and how do you explain the fact that it is different from $2$?**

In [7]:
(define (next n)
  (if (= n 2) 3 (+ n 2)))

(define (smallest-divisor-o n) (find-divisor-o n 2))
(define (find-divisor-o n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor-o n (next test-divisor)))))

(define (prime-o? n)
  (= n (smallest-divisor-o n)))

(python-exec 
"
import time
def runtime():
  return int(round(time.time() * 1000000))
")
(define (square n) (* n n))
(define (divides? a b) (= (remainder b a) 0))

(define (timed-prime-test-o n)
  (newline)
  (display n)
  (start-prime-test-o n (runtime)))
(define (start-prime-test-o n start-time)
  (if (prime-o? n)
      (report-prime-o (- (runtime) start-time))))
(define (report-prime-o elapsed-time)
  (display " *** ")
  (display elapsed-time))

In [9]:
(timed-prime-test-o 1009)


1009 *** 16143

In [10]:
(timed-prime-test-o 1013)


1013 *** 24077

In [11]:
(timed-prime-test-o 1019)


1019 *** 21269

In [12]:
(timed-prime-test-o 10007)


10007 *** 55020

In [13]:
(timed-prime-test-o 10009)


10009 *** 50183

In [14]:
(timed-prime-test-o 10037)


10037 *** 52375

In [15]:
(timed-prime-test-o 100003)


100003 *** 182760

In [16]:
(timed-prime-test-o 100019)


100019 *** 169948

In [17]:
(timed-prime-test-o 100043)


100043 *** 164158

In [18]:
(timed-prime-test-o 1000003)


1000003 *** 388764

In [19]:
(timed-prime-test-o 1000033)


1000033 *** 389255

In [20]:
(timed-prime-test-o 1000037)


1000037 *** 397710

The following table shows the times taken by the timed-prime-test procedures and their ratios: 

| Prime     | smallest-divisor  | Optimized smallest-divisor    | Ratio |
|-----------|-------------------|-------------------------------|-------|
| 1009      | 34456             | 16143                         | 2.134 |
| 1013      | 40018             | 24077                         | 1.662 |
| 1019      | 31267             | 21269                         | 1.470 |
| 10007     | 91252             | 55020                         | 1.658 |
| 10009     | 108091            | 50183                         | 2.154 |
| 10037     | 90199             | 52375                         | 1.722 |
| 100003    | 324643            | 182760                        | 1.776 |
| 100019    | 379223            | 169949                        | 2.231 |
| 100043    | 336446            | 164158                        | 2.049 |
| 1000003   | 797337            | 388764                        | 2.051 |
| 1000033   | 996661            | 389255                        | 2.560 |
| 1000037   | 844889            | 397710                        | 2.124 |


As we can see from the table, the ratio of times is not always $2$ or more, and that is especially true for lower prims. This can be attributed to the fact that even though the optimized `smallest-divisor` tests only half the numbers as the unoptimized one does, it makes a call to the procedure `next` which performs an equality test and addition. So the steps added due to the evaluation of `next` counteract the reduction in steps due to our optimization.

### Exercise 1.24

**Modify the `timed-prime-test` procedure of Exercise 1.22 to use `fast-prime?` (the Fermat method), and test each of the 12 primes you found in that exercise. Since the Fermat test has $\Theta(\log{n})$ growth, how would you expect the time to test primes near 1,000,000 to compare with the time needed to test primes near 1000? Do your data bear this out? Can you explain any discrepancy you find?**

In [3]:
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp) (remainder (square (expmod base (/ exp 2) m)) m))
        (else (remainder (* base (expmod base (- exp 1) m)) m))))
(define (fermat-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1)))))
(define (fast-prime? n times)
  (cond ((= times 0) #t)
        ((fermat-test n) (fast-prime? n (- times 1)))
        (else #f)))

In [18]:
(python-exec 
"
import time
def runtime():
  return int(round(time.time() * 1000000))
")
(define (square n) (* n n))

(define (timed-prime-test-fast n)
  (newline)
  (display n)
  (start-prime-test-fast n (runtime)))
(define (start-prime-test-fast n start-time)
  (if (fast-prime? n 10)
      (report-prime (- (runtime) start-time))))
(define (report-prime elapsed-time)
  (display " *** ")
  (display elapsed-time))

In [20]:
(timed-prime-test-fast 1009)


1009 *** 145055

In [21]:
(timed-prime-test-fast 1013)


1013 *** 125862

In [22]:
(timed-prime-test-fast 1019)


1019 *** 117172

In [23]:
(timed-prime-test-fast 10007)


10007 *** 146663

In [24]:
(timed-prime-test-fast 10009)


10009 *** 159042

In [25]:
(timed-prime-test-fast 10037)


10037 *** 150413

In [26]:
(timed-prime-test-fast 100003)


100003 *** 201457

In [27]:
(timed-prime-test-fast 100019)


100019 *** 162038

In [28]:
(timed-prime-test-fast 100043)


100043 *** 169991

In [29]:
(timed-prime-test-fast 1000003)


1000003 *** 198246

In [31]:
(timed-prime-test-fast 1000033)


1000033 *** 208288

In [32]:
(timed-prime-test-fast 1000037)


1000037 *** 220590

On average, testing for primes near 1000 took 129363 'time units' and testing for primes near 1000000 took 209041.3 'time units'. The ratio is just $1.616$. Since $\log{1000000}/\log{1000}$ is $2$, the observed ratio is somewhat lower than we expected. This could be because here we are applying the fermat test on each input 10 times, and the constant (independent of input size) number of steps required for procedure application and book-keeping could be causing the ratio be smaller.

### Exercise 1.25

**Alyssa P. Hacker complains that we went to a lot of extra work in writing expmod. After all, she says, since we already know how to compute exponentials, we could have simply written**

```scheme
(define (expmod base exp m)
  (remainder (fast-expt base exp) m))
```

**Is she correct? Would this procedure serve as well for our fast prime tester? Explain.**

Our `expmod` procedure works by exploiting the fact that the $mod$ operation is multiplicative, that is, $ab\mod n = (a\mod n)(b\mod n)\mod n$. This lets us avoid having to work with very large numbers when computing $a^m\mod n$, because we don't have to compute $a^m$ first. However, the procedure suggested by Alyssa P. Hacker is doing exactly that - it is computing $a^m$ first and calculating the remainder with $m$. The procedure produces the correct result, and the number of steps are more or less the same, but because operations with large numbers actually takes a non-constant number of steps internally, the suggested procedure would be slower with large numbers. That weakness will be particularly visible with our fast prime tester, because we are repeatedly applying the `expmod` operation with very large powers.

### Exercise 1.26

**Louis Reasoner is having great difficulty doing Exercise 1.24. His `fast-prime?` test seems to run more slowly than his `prime?` test. Louis calls his friend Eva Lu Ator over to help. When they examine Louis’s code, they find that he has rewritten the `expmod` procedure to use an explicit multiplication, rather than calling square:**

```scheme
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp)
         (remainder (* (expmod base (/ exp 2) m)
                       (expmod base (/ exp 2) m))
                    m))
        (else
         (remainder (* base
                       (expmod base (- exp 1) m))
                    m))))
```

**"I don’t see what difference that could make", says Louis. "I do." says Eva. "By writing the procedure like that, you have transformed the $\Theta(\log{n})$ process into a $\Theta(n)$ process." Explain.**

By replacing the `square` application with an explicit multiplication, Louis has ensured that `expmod` will be called two times in that part of the procedure. If he had left the `square` call, the evaluator would have evaluated the `expmod` call first and squared the resulting value. Writing it this way, the multiplication procedure has to evaluate its arguments, of which there are two - the two calls to `expmod`. In our original procedure, a $\Theta(\log{n})$ order of growth was achieved by halving the exponent in that step. Because the recursion terminates when the exponent is zero, and we can only halve the exponent $\log_2{n}$ times, a logarithmic order of growth was achieved. But now we are halving the exponent but also doubling the number of calls, and those effects cancel each other out. The second `cond` case where `exp` is even does not affect the number of steps now, only the third case does, which subtracts 1 from `exp`. Therefore, the process now has $\Theta(n)$ order of growth.

### Exercise 1.27

**Demonstrate that the Carmichael numbers listed in Footnote 1.47 really do fool the Fermat test. That is, write a procedure that takes an integer $n$ and tests whether an is congruent to $a$ modulo $n$ for every $a \lt n$, and try your procedure on the given Carmichael numbers.**

In [6]:
(define (square n) (* n n))
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp) (remainder (square (expmod base (/ exp 2) m)) m))
        (else (remainder (* base (expmod base (- exp 1) m)) m))))
(define (carmichael-test n)
  (define (display-carmichael)
    (display "Carmichael number"))
  (define (display-not-carmichael)
    (display "Not a Carmichael number"))
  (define (congruence-test a)
    (cond ((= a 1) (display-carmichael))
          ((= (expmod a n n) a) (congruence-test (- a 1)))
          (else (display-not-carmichael))))
  (congruence-test (- n 1)))

In [9]:
(carmichael-test 561)

Carmichael number

In [10]:
(carmichael-test 1105)

Carmichael number

In [11]:
(carmichael-test 1729)

Carmichael number

In [12]:
(carmichael-test 2465)

Carmichael number

In [13]:
(carmichael-test 2821)

Carmichael number

In [14]:
(carmichael-test 6601)

Carmichael number

### Exercise 1.28

**One variant of the Fermat test that cannot be fooled is called the *Miller-Rabin test* (Miller 1976; Rabin 1980). This starts from an alternate form of Fermat’s Little Theorem, which states that if $n$ is a prime number and $a$ is any positive integer less than $n$, then $a$ raised to the $(n-1)$-st power is congruent to 1 modulo $n$. To test the primality of a number $n$ by the Miller-Rabin test, we pick a random number $a \lt n$ and raise $a$ to the $(n-1)$-st power modulo $n$ using the `expmod` procedure. However, whenever we perform the squaring step in `expmod`, we check to see if we have discovered a "nontrivial square root of 1 modulo $n$", that is, a number not equal to 1 or $n-1$ whose square is equal to 1 modulo $n$. It is possible to prove that if such a nontrivial square root of 1 exists, then $n$ is not prime. It is also possible to prove that if $n$ is an odd number that is not prime, then, for at least half the numbers $a < n$, computing $a^{n-1}$ in this way will reveal a nontrivial square root of 1 modulo $n$. (This is why the Miller-Rabin test cannot be fooled.) Modify the `expmod` procedure to signal if it discovers a nontrivial square root of 1, and use this to implement the Miller-Rabin test with a procedure analogous to `fermat-test`. Check your procedure by testing various known primes and non-primes. Hint: One convenient way to make `expmod` signal is to have it return 0.**

In [1]:
(define (square n) (* n n))
(define (expmod base exp m)
  (define (test-root-of-one n)
    (if (and (not (= n 1)) (not (= n (- m 1))) (= (remainder (square n) m) 1))
        0
        (remainder (square n) m)))
  (cond ((= exp 0) 1)
        ((even? exp) (test-root-of-one (expmod base (/ exp 2) m)))
        (else (remainder (* base (expmod base (- exp 1) m)) m))))
(define (miller-rabin-test n)
  (define (try-it a)
    (= (expmod a n n) a))
  (try-it (+ 1 (random (- n 1)))))

In [2]:
(miller-rabin-test 6601)

#f

In [3]:
(miller-rabin-test 100003)

#t

In [4]:
(miller-rabin-test 482683)

#t

In [8]:
(miller-rabin-test 2465)

#f

### Exercise 1.29

**Simpson’s Rule is a more accurate method of numerical integration than the method illustrated above. Using Simpson’s Rule, the integral of a function $f$ between $a$ and $b$ is approximated as**

$$
\frac{h}{3}(y_0 + 4y_1 + 2y_2 + 4y_3 + 2y_4 + \cdots + 2y_{n-2} + 4y_{n-1} + y_n),
$$

**where $h = (b - a)/n$, for some even integer $n$, and $y_k = f(a+kh)$. (Increasing $n$ increases the accuracy of the approximation.) Define a procedure that takes as arguments $f$ , $a$, $b$, and $n$ and returns the value of the integral, computed using Simpson’s Rule. Use your procedure to integrate cube between $0$ and $1$ (with $n = 100$ and $n = 1000$), and compare the results to those of the integral procedure shown above.**

In [9]:
(define (simpsons-rule f a b n)
  (define (y k h)
    (f (+ a (* k h))))
  (define (sum-iter k sum h)
    (if (> k n)
        sum
        (sum-iter (+ k 1) (+ sum (* (y k h)
                  (cond ((or (= k 0) (= k n)) 1)
                        ((even? k) 2)
                        (else 4)))) h)))
  (* (/ (- b a) (* 3 n)) (sum-iter 0 0 (/ (- b a) n))))
  

In [14]:
(define (cube x) (* x x x))
(simpsons-rule cube 0 1.0 100)

0.25000000000000006

In [15]:
(simpsons-rule cube 0 1.0 1000)

0.25000000000000006

As we see in the results above, output values produced by procedure implementing Simpson's rule is much more accurate than those produced by the procedure given in the text. Simpson's rules produces essentially the same value ($0.25000000000000006$) with $n=100$ and $n=1000$. This is practically equal (discounting the error produced by limited precision floating point arithmetic) to the true value $0.25$, whereas the given procedure produced $0.24999987500001$ with a step size of $0.001$, corresponding to $1000$ steps.

### Exercise 1.30

**The `sum` procedure above generates a linear recursion. The procedure can be rewritten so that the sum is performed iteratively. Show how to do this by filling in the missing expressions in the following definition:**

```scheme
(define (sum term a next b)
  (define (iter a result)
    (if ⟨??⟩
        ⟨??⟩
        (iter ⟨??⟩ ⟨??⟩)))
  (iter ⟨??⟩ ⟨??⟩))
```

The procedure given below performs the summation iteratively:

In [22]:
(define (sum term a next b)
  (define (iter a result)
    (if (> a b)
        result
        (iter (next a) (+ result (term a)))))
  (iter a 0))

In [23]:
(define (square x) (* x x))
(define (inc n) (+ n 1))
(sum square 1 inc 10)

385

In [25]:
(define (identity x) x)
(sum identity 1 inc 10)

55

### Exercise 1.31

**a. The `sum` procedure is only the simplest of a vast number of similar abstractions that can be captured as higher-order procedures. Write an analogous procedure called `product` that returns the product of the values of a function at points over a given range. Show how to define `factorial` in terms of `product`. Also use product to compute approximations to $\pi$ using the formula**

$$
\frac{\pi}{4} = \frac{2\cdot4\cdot4\cdot6\cdot6\cdot8\cdots}{3\cdot3\cdot5\cdot5\cdot7\cdot7\cdots}
$$

**b. If your product procedure generates a recursive process, write one that generates an iterative process. If it generates an iterative process, write one that generates a recursive process.**

a.

In [1]:
(define (product term a next b)
  (if (> a b)
      1
      (* (term a) (product term (next a) next b))))

(define (factorial n)
  (define (inc m) (+ m 1))
  (define (id m) m)
  (product id 1 inc n))
(factorial 5)

120

In [11]:
(define (pi-approx n)
  (define (term k)
    (/ (* 2 (+ (quotient k 2) 1)) (+ (* 2 (+ (quotient (- k 1) 2) 1)) 1)))
  (define (inc k) (+ k 1))
  (/ (* 4 (product term 1 inc n)) 1.0))    

In [12]:
(pi-approx 3)

2.8444444444444446

In [13]:
(pi-approx 10)

3.2751010413348074

In [14]:
(pi-approx 100)

3.1570301764551676

In [15]:
(pi-approx 10000)

3.1417497057380523

b. The `product` procedure given above generates a linear recursive process. The `iproduct` procedure given below generates an iterative process.

In [21]:
(define (iproduct term a next b)
  (define (product-iter t prod)
    (if (> t b)
        prod
        (product-iter (next t) (* (term t) prod))))
  (product-iter a 1))

In [22]:
(define (factorial n)
  (define (inc m) (+ m 1))
  (define (id m) m)
  (iproduct id 1 inc n))
(factorial 6)

720